<a href="https://colab.research.google.com/github/daisysong76/AI-LLM-Computer-vision/blob/main/text_to_video_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install diffusers transformers torch accelerate

# Install OpenCV for image processing
!pip install opencv-python

# Install Pillow for image handling
!pip install Pillow

# Install ffmpeg-python for video creation
!pip install ffmpeg-python

# Install basicsr and realesrgan for super-resolution
!pip install basicsr
!pip install realesrgan
!pip install torch torchvision --upgrade
# Install diffusers, transformers, torch, and accelerate

# Download and install FFmpeg for video processing
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Install necessary libraries
!pip install diffusers transformers torch accelerate opencv-python Pillow ffmpeg-python basicsr realesrgan

# Install ffmpeg for video creation
!apt-get install ffmpeg


1.  Text Embeddings and Scene Understanding

For this demo, we'll skip complex text embeddings and scene graph extractions and go directly to generating images using Stable Diffusion. If you want to extend this later, you can add the text processing steps for embeddings.

In [3]:
from transformers import pipeline

# Use a pre-trained transformer model for text embeddings
nlp = pipeline("feature-extraction", model="bert-base-uncased")

# Example input text
text = "A person is walking in a park on a sunny day."

# Generate text embeddings (we won't use this in the demo but it's here for scene graph extension)
text_embeddings = nlp(text)
print("Text Embeddings:", text_embeddings)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Text Embeddings: [[[0.12956957519054413, -0.3371562063694, -0.45902350544929504, 0.38899606466293335, 0.09469199925661087, -0.1555604487657547, 0.038663845509290695, 1.3458236455917358, -0.4186001420021057, -0.6484197378158569, -0.07988616079092026, -0.6214295625686646, -0.051610175520181656, 0.61405348777771, 0.4042811393737793, 0.4363325536251068, 0.365595281124115, 0.35438647866249084, 0.13176840543746948, 0.3050898313522339, -0.16838189959526062, 0.1557198017835617, -0.19369666278362274, -0.4478496313095093, 0.030805304646492004, -0.5357741117477417, 0.3123336434364319, -0.09347675740718842, 0.33348342776298523, -0.182814821600914, -0.7312455177307129, 0.09640485793352127, -0.9077777862548828, -1.00961172580719, 0.44411951303482056, 0.07773685455322266, 0.20238764584064484, -0.27129295468330383, -0.25600430369377136, 0.10496202111244202, -0.8228533267974854, 0.008455605246126652, 0.23425093293190002, -0.061802808195352554, -0.08849789202213287, -0.7447344660758972, -2.5726876258850

In [4]:
from diffusers import StableDiffusionPipeline
import os
from PIL import Image

# Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

# Create a directory to store the generated frames
os.makedirs('data/keyframes', exist_ok=True)

# Base text prompt
base_prompt = "A person walking in a park on a sunny day"

# Generate 15 images with slight variations in prompt
for i in range(15):
    prompt = base_prompt + f", frame {i+1}"
    image = pipe(prompt).images[0]

    # Save each generated image
    image.save(f"data/keyframes/frame_{i+1:03d}.png")

print("Generated 15 keyframes.")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Generated 15 keyframes.


In [5]:
# Use FFmpeg to create a video from the generated frames
!ffmpeg -r 24 -i data/keyframes/frame_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p output_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# Advanced Approach
1.Text Embeddings and Scene Graph Extraction
For processing text input, you can use models like GPT or BERT to extract embeddings and a scene graph from a textual description.

In [2]:
from transformers import pipeline

# Example: Using GPT-2 or BERT for text embeddings
text = "A person is walking in a park on a sunny day."
nlp = pipeline("feature-extraction", model="bert-base-uncased")

# Get text embeddings
text_embeddings = nlp(text)
print(text_embeddings)

# For scene graph extraction (e.g., using Spacy for NER and dependency parsing)
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Extract entities and relationships
for ent in doc.ents:
    print(ent.text, ent.label_)

# For more advanced scene graph processing, consider using libraries like Graph4NLP


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is 

[[[0.12956957519054413, -0.3371562063694, -0.45902350544929504, 0.38899606466293335, 0.09469199925661087, -0.1555604487657547, 0.038663845509290695, 1.3458236455917358, -0.4186001420021057, -0.6484197378158569, -0.07988616079092026, -0.6214295625686646, -0.051610175520181656, 0.61405348777771, 0.4042811393737793, 0.4363325536251068, 0.365595281124115, 0.35438647866249084, 0.13176840543746948, 0.3050898313522339, -0.16838189959526062, 0.1557198017835617, -0.19369666278362274, -0.4478496313095093, 0.030805304646492004, -0.5357741117477417, 0.3123336434364319, -0.09347675740718842, 0.33348342776298523, -0.182814821600914, -0.7312455177307129, 0.09640485793352127, -0.9077777862548828, -1.00961172580719, 0.44411951303482056, 0.07773685455322266, 0.20238764584064484, -0.27129295468330383, -0.25600430369377136, 0.10496202111244202, -0.8228533267974854, 0.008455605246126652, 0.23425093293190002, -0.061802808195352554, -0.08849789202213287, -0.7447344660758972, -2.5726876258850098, -0.113496609

2. Text-to-Image Generation using DALL-E or Stable Diffusion
To generate key frames from textual descriptions, you can leverage Stable Diffusion or DALL-E. The Hugging Face library provides easy-to-use interfaces for such models.

In [3]:
from diffusers import StableDiffusionPipeline

# Load the Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

# Generate an image from text
prompt = "A person walking in a park on a sunny day"
image = pipe(prompt).images[0]

# Save the image
image.save("generated_frame.png")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

3. Temporal Consistency: Frame Interpolation with Vid2Vid
Once you have the key frames, you need to interpolate intermediate frames to ensure smooth transitions. This can be done using frame interpolation methods such as Vid2Vid or Dain-App.

an implementation of Vid2Vid here: Vid2Vid GitHub Repository.

In [6]:
# Clone the repository
!git clone https://github.com/NVIDIA/vid2vid.git

# Follow the installation instructions
# You'll need to provide the generated frames as input for interpolation


Cloning into 'vid2vid'...
remote: Enumerating objects: 1030, done.
remote: Total 1030 (delta 0), reused 0 (delta 0), pack-reused 1030 (from 1)
Receiving objects: 100% (1030/1030), 168.36 MiB | 26.99 MiB/s, done.
Resolving deltas: 100% (279/279), done.


In [7]:
cd vid2vid

/content/vid2vid


Alternatively, you can use DAIN for deep video frame interpolation: DAIN GitHub Repository.

4. Post-processing: Super-Resolution with ESRGAN
For improving the resolution of the generated frames, you can use ESRGAN (Enhanced Super-Resolution GAN).

In [ ]:
# Clone the ESRGAN repository
!git clone https://github.com/xinntao/ESRGAN.git
cd ESRGAN

# Follow the installation steps and run the inference script on the generated frames


In [10]:
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

# Initialize the model
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
upsampler = RealESRGANer(scale=4, model_path='ESRGAN.pth', model=model)

# Load and upscale image
image = cv2.imread('generated_frame.png')
upscaled_image = upsampler.enhance(image)
cv2.imwrite('upscaled_frame.png', upscaled_image)


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

5. Video Synthesis: Combine Frames with FFmpeg
Once you have the upscaled and interpolated frames, you can use FFmpeg to combine them into a video.

In [ ]:
# Assuming your frames are saved as frame_001.png, frame_002.png, ...
ffmpeg -r 24 -i frame_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p output_video.mp4


Reinforcement Learning for Optimization
You could incorporate RL to fine-tune the video generation process. This requires a custom environment setup using frameworks like Stable-Baselines3.